In [4]:
import joblib
import numpy as np
import matplotlib.pyplot as plt
import arviz as az
from cmdstanpy import CmdStanModel

# === Wczytanie danych ===
X, y, purpose = joblib.load("processed_data.pkl")
if hasattr(X, 'toarray'):
    X = X.toarray()

X = X.astype(np.float32)
y = y.astype(np.int32)
N, K = X.shape

stan_data = {
    "N": N,
    "K": K,
    "X": X.tolist(),
    "y": y.tolist()
}

# === PRIOR PREDICTIVE CHECK – DANE (STAN) ===
ppc_model = CmdStanModel(stan_file="model1_ppc.stan")
ppc_fit = ppc_model.sample(
    data={"N": N, "K": K, "X": X.tolist()},
    chains=1,
    iter_sampling=1000,
    iter_warmup=1,
    fixed_param=True,
    seed=42
)

ppc_df = ppc_fit.draws_pd()
y_sim_matrix = ppc_df[[col for col in ppc_df.columns if "y_sim" in col]].values
mean_simulated_defaults = y_sim_matrix.mean(axis=1)

plt.hist(mean_simulated_defaults, bins=30, edgecolor='k')
plt.title("Prior predictive (Stan): mean(defaults)")
plt.xlabel("Średnia liczba defaultów w próbie")
plt.show()

# === MODELOWANIE WŁAŚCIWE ===
model = CmdStanModel(stan_file="model1.stan")
fit = model.sample(
    data=stan_data,
    chains=4,
    iter_sampling=1000,
    iter_warmup=1000,
    seed=42
)

idata = az.from_cmdstanpy(posterior=fit)

summary = az.summary(idata, var_names=["alpha", "beta"])
print(summary[["r_hat", "ess_bulk", "ess_tail"]])
print("Divergences:", int(idata.sample_stats["diverging"].sum()))

az.plot_trace(idata, var_names=["alpha", "beta"], compact=True)
plt.show()

az.plot_posterior(
    idata,
    var_names=["alpha", "beta"],
    coords={"beta_dim_0": list(range(5))},  # jeśli chcesz pokazać tylko kilka
    hdi_prob=0.95
)
plt.show()

# === PREDYKCJA ===
posterior_means = idata.posterior.mean(dim=("chain", "draw"))
alpha_mean = posterior_means["alpha"].values.item()
beta_mean = posterior_means["beta"].values

logits = alpha_mean + X @ beta_mean
probs = 1 / (1 + np.exp(-logits))

plt.hist(probs, bins=30, edgecolor='k')
plt.title("Rozkład przewidywanych P(default)")
plt.xlabel("P(default)")
plt.ylabel("Liczba obserwacji")
plt.show()

# === KLASYFIKACJA ===
y_pred = (probs > 0.5).astype(int)
acc = (y_pred == y).mean()
print(f"Dokładność klasyfikacji: {acc:.2f}")

plt.hist(y, bins=2, alpha=0.5, label="Rzeczywiste", edgecolor='k')
plt.hist(y_pred, bins=2, alpha=0.5, label="Predykowane", edgecolor='k')
plt.xticks([0, 1])
plt.legend()
plt.title("Rzeczywiste vs Predykowane etykiety")
plt.show()


ValueError: Failed to get source info for Stan model '/workspace/model1_ppc.stan'. Console:
Semantic error in '/workspace/model1_ppc.stan', line 9, column 2 to column 36:
   -------------------------------------------------
     7:  generated quantities {
     8:    real alpha = normal_rng(0, 5);
     9:    vector[K] beta = normal_rng(0, 2);
           ^
    10:    array[N] int y_sim;
    11:  
   -------------------------------------------------

Ill-typed arguments supplied to assignment operator =:
The left hand side has type
  vector
and the right hand side has type
  real
